In [4]:
import requests
from bs4 import BeautifulSoup
import re
import sys
from time import sleep

In [2]:
headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

parms = {
    "pagenum" : 1,
    "Depth" : 3111
}

In [3]:
url = "http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConven.jsp"

In [5]:
def getUrl(url, parameter=None, num_retries=2, select=None):
    html = requests.get(url, params=parameter, headers=headers)
    
    if 500 <= html.status_code  < 600 and num_retries > 0:
        print(html.status_code, html.reason)
        return getUrl(url, params, num_retries=1)
        
    dom = BeautifulSoup(html.text, "lxml")
    links = dom.select(select)
    
    return links

In [6]:
select_css = { 'place': 'ul.travel-list2 > li > dl > dt > a ',
               'address' : 'ul.travel-list2 > li > dl > dd > p',
               'phone' : '.tel',
               'facility': 'ul.travel-list2 > li > dl > dd > ul.facil' }

In [7]:
place_Depth_code = {
    "korean" : 3111,
    "western" : 3211,
    "chinese"  : 3311,
    "japanese" : 3411,
    "flour_based" : 3511,
    "other" : 3611,
}

# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  

In [8]:
korean = []
western = []
chinese = []
japanese = []
flour_based = []
other = []

In [9]:
# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  
# type 구조 [{ }] 리스안에 딕셔너리 구조 
final_dict = {
'korean' : korean ,
'western' : western ,
'chinese' : chinese ,
'japanese' : japanese ,
'flour_based' : flour_based ,
'other' : other,

}

info_dict = {}

In [10]:
for place in place_Depth_code:
    place_code = place_Depth_code[place]
    parms['Depth'] = place_code
    result = []  
    while True:
        for select in select_css:
            get_info = getUrl(url, parameter = parms, select = select_css[select])
            # 장소, 장소_사이트, 주소, 전화번호, 편의시설
            if 'place' is select:
                place_name = [info.text for info in get_info] # 장소
                place_href = [info['href'] for info in get_info] # 장소_사이트
            if 'address' is select:
                address_pre = [re.sub(u'\xa0 \r\n\t*',"",info.text) for info in get_info] # 주소
                address =[re.sub(u'주소 :  ',"",info) for info in address_pre]
            if 'phone' is select:
                phone = [info.text for info in get_info] # 전화번호
            if 'facility' is select:
                facility_pre_select = [info for info in get_info] # 편의시설
                
        if get_info: 
            for info_num in range(0, len(place_name)):
                print(place_name[info_num])
                facility_select =  facility_pre_select[info_num].select('li > img')
                facility_select = [row_facility['alt'] for row_facility in facility_select]
                print(facility_select)
                info_dict = {'place_name' : place_name[info_num],
                             'place_href' : 'http://disability.seoul.go.kr'+place_href[info_num],
                             'place_address' : address[info_num],
                             'place_facility' : facility_select
                             }  #     #
                result.append(info_dict)
            print(info_dict)
            parms['pagenum'] = parms['pagenum'] + 1
            # pagenum ->for next page
        else:
            parms['pagenum'] = 1
            break
    final_dict[place] = result

FR푸드시스템
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
OK목장
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
The만나
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
㈜이리오어라(한정식)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
가야옥
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '

근린생활시설(벽창우생고기전문점)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
근린생활시설(봉평산골메밀촌)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
근린생활시설(생고기전문점취어당)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
근린생활시설(엉터리생고기)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
근린생활시설(진할매칼국수보쌈)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가

뚝심
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
마루갈비
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
만가화로구이
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
만석장
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
맛있는고기구워먹는곳
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인

삼수갑산
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
삼호가든
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
삼호복집
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
상도갈비
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
새마을식당
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서

양대박
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
양주골오리마을
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
양지촌
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
양촌리
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
양촌리
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스

...

한국맥도날드
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
한국맥도날드송정역
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
한국피자헛
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
한국피자헛등촌점(샷뽀르)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
화수목레스토랑
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표

아사달횟집
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
아지코우보
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
어부박씨
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
이춘복참치
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
일반음식점(삿뽀로)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시

㈜젠 하이더 웨이-동남아음식
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
㈜청담안
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
갤러리 시간
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
골든너겟
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
곰씨네(씨크랩건물)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용불가', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소

사이공 스픈 32-동남아음식
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
샬롬빌
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
서광폴리머
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
석천(근린생활시설)
['주출입구접근로 이용불가', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
세븐몽키스(금원빌딩)
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가',

진미파라곤
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
청산유통
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
카페 무이무이
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
카페뎀셀브쥬
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용불가', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장애인 편의서비스 이용불가', '청각각장애인 편의서비스 이용불가', '안내 서비스 이용불가', '휠체어 대여 이용불가']
카페프렌즈
['주출입구접근로 이용가능', '장애인 전용 주차구역 이용가능', '주출입구 높이차이제거 이용가능', '장애인용 승강기 이용불가', '장애인용 화장실 이용불가', '장애인용 객실 이용불가', '장애인용 관람석 이용불가', '매표소 이용불가', '시각장

In [11]:
number_len =0 
for place in place_Depth_code:
    number_len += len(final_dict[place])
print(number_len)

414


In [12]:
facility_select =  "div.imgIcoBview dl.accordion" # 필요한 부분만 자르기

In [13]:
facility_available_name_select = "dt > a" # 시설물 설치된 이름
facility_is_available_select = "dt > span" # 시설물 이용가능 or 이용 불편 
facility_info_select = "ul.bulListDot > li" # 시설물 상세 정보 

In [14]:
for place in place_Depth_code:
    info_result = [] 
    print(place)
    for i, info in enumerate(final_dict[place]):
        info_result.clear()
        get_info = getUrl(info['place_href'], select = facility_select)
        facility_available_name= [info.text for info in get_info[0].select(facility_available_name_select)]
        facility_is_available = [info.text for info in get_info[0].select(facility_is_available_select)]
        facility_info = [info for info in get_info[0].select(facility_info_select)]
        facility_info = [info.__str__() for info in facility_info]
        facility_info = [ re.sub('^<li>|-|(<br/>)?</li>$' , '' , info).split('<br/>') for info in facility_info]
    
        for info_num in range(0, len(facility_available_name)):
            info_dict = {'facility_available_name' : facility_available_name[info_num],
                     'facility_is_available' : facility_is_available[info_num],
                     'facility_info' : facility_info[info_num]
                     }
            info_result.append(info_dict)
        final_dict[place][i]['facility_detail_info'] = info_result

korean
western
chinese
japanese
flour_based
other


In [15]:
import json
import urllib.request
import urllib.request
import datetime
import time
import json

In [16]:
def get_request_url(url):
    
    client_id="R63uNWNQiaTp1EeZl7bW"
    client_secret = "1Im1JohssC"
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getGeoData(address):
    
    base = "https://openapi.naver.com/v1/map/geocode"
    node = ""
    parameters = "?query=%s" % urllib.parse.quote(address)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [17]:
for place in place_Depth_code:
    info_result = [] 
    print(place)
    
    for info_num,info in enumerate(final_dict[place]):
        print(info['place_address'])
        jsonResult = getGeoData(info['place_address'])
        
        try:
            for item in jsonResult['result']['items']:
                    final_dict[place][info_num]['lat'] =  str(item['point']['y'])
                    final_dict[place][info_num]['lon'] =  str(item['point']['x'])
        except Exception as e:
            print("예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            final_dict[place][info_num]['lat'] = "NaN"
            final_dict[place][info_num]['lon']= "NaN"    
            print(e)

korean
서울특별시 강서구 공항대로  318
[2018-08-03 00:48:50.760292] Url Request Success
서울특별시 송파구 백제고분로 547
HTTP Error 404: Not Found
[2018-08-03 00:48:50.849041] Error for URL : https://openapi.naver.com/v1/map/geocode?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%86%A1%ED%8C%8C%EA%B5%AC%20%EB%B0%B1%EC%A0%9C%EA%B3%A0%EB%B6%84%EB%A1%9C%20547
예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'NoneType' object is not subscriptable
서울특별시 노원구 석계로 103
[2018-08-03 00:48:51.071483] Url Request Success
서울특별시 강남구 언주로153길  12
[2018-08-03 00:48:51.178164] Url Request Success
서울특별시 도봉구 도당로   2
[2018-08-03 00:48:51.296846] Url Request Success
서울특별시 영등포구 가마산로 330
[2018-08-03 00:48:51.388634] Url Request Success
서울특별시 노원구 덕릉로 692
[2018-08-03 00:48:51.491328] Url Request Success
서울특별시 서초구 강남대로  204
[2018-08-03 00:48:51.596043] Url Request Success
서울특별시 구로구 경인로 281
[2018-08-03 00:48:51.703756] Url Request Success
서울특별시 도봉구 노해로 259
[2018-08-03 00:48:51.858695] Url Request Success
서울특별시 강남구 언주로 855
[2018-08-03 00

[2018-08-03 00:49:02.877868] Url Request Success
서울특별시 강남구 도곡로63길  20
[2018-08-03 00:49:02.975607] Url Request Success
서울특별시 강서구 양천로 608
[2018-08-03 00:49:03.076375] Url Request Success
서울특별시 강남구 압구정로75길  6
[2018-08-03 00:49:03.173135] Url Request Success
서울특별시 용산구 회나무로  75
[2018-08-03 00:49:03.303775] Url Request Success
서울특별시 강동구 양재대로85길 16
[2018-08-03 00:49:03.407491] Url Request Success
서울특별시 강남구 도산대로 311
[2018-08-03 00:49:03.504194] Url Request Success
서울특별시 강남구 영동대로 725
[2018-08-03 00:49:03.601124] Url Request Success
서울특별시 송파구 문정로4길  14-38
HTTP Error 404: Not Found
[2018-08-03 00:49:03.690695] Error for URL : https://openapi.naver.com/v1/map/geocode?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%86%A1%ED%8C%8C%EA%B5%AC%20%EB%AC%B8%EC%A0%95%EB%A1%9C4%EA%B8%B8%20%2014-38
예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'NoneType' object is not subscriptable
서울특별시 중구 동호로14길 18
[2018-08-03 00:49:03.858249] Url Request Success
서울특별시 중구 명동4길 25
[2018-08-03 00:49:03.984908] Url Requ

[2018-08-03 00:49:15.200908] Url Request Success
서울특별시 강남구 언주로168길  16
[2018-08-03 00:49:15.399379] Url Request Success
서울특별시 도봉구 시루봉로 68
[2018-08-03 00:49:15.539005] Url Request Success
서울특별시 도봉구 도봉로180길 6
[2018-08-03 00:49:15.637741] Url Request Success
서울특별시 은평구 응암로 229
[2018-08-03 00:49:15.754465] Url Request Success
서울특별시 강동구 양재대로 1651
[2018-08-03 00:49:15.857153] Url Request Success
서울특별시 서대문구 연세로 10
[2018-08-03 00:49:16.004759] Url Request Success
서울특별시 도봉구 노해로  285
[2018-08-03 00:49:16.146381] Url Request Success
서울특별시 강남구 강남대로  242
[2018-08-03 00:49:16.287003] Url Request Success
서울특별시 성북구 도봉로 1
[2018-08-03 00:49:16.383745] Url Request Success
서울특별시 강남구 도산대로  334
[2018-08-03 00:49:16.813598] Url Request Success
서울특별시 강남구 도산대로45길 18-2
[2018-08-03 00:49:16.916321] Url Request Success
서울특별시 용산구 이태원로 243
[2018-08-03 00:49:17.029018] Url Request Success
서울특별시 강남구 압구정로60길 18
[2018-08-03 00:49:17.314257] Url Request Success
서울특별시 은평구 진흥로 87
[2018-08-03 00:49:17.517714] Url Request Su

[2018-08-03 00:49:28.918222] Url Request Success
서울특별시 성동구 고산자로26길 12
[2018-08-03 00:49:29.025933] Url Request Success
서울특별시 종로구 대학로10길 5
[2018-08-03 00:49:29.129657] Url Request Success
서울특별시 강서구 공항대로46길  90
[2018-08-03 00:49:29.256321] Url Request Success
서울특별시 은평구 연서로  165
[2018-08-03 00:49:29.369023] Url Request Success
서울특별시 마포구 어울마당로 147-1
[2018-08-03 00:49:29.480722] Url Request Success
서울특별시 강동구 강동대로53길 22
[2018-08-03 00:49:29.599400] Url Request Success
서울특별시 강북구 도봉로10가길 12
[2018-08-03 00:49:29.735036] Url Request Success
서울특별시 금천구 범안로  1203
[2018-08-03 00:49:29.846739] Url Request Success
서울특별시 강남구 선릉로158길 13-16
[2018-08-03 00:49:29.943480] Url Request Success
서울특별시 강남구 도산대로  426
[2018-08-03 00:49:30.059220] Url Request Success
서울특별시 서대문구 응암로  103
[2018-08-03 00:49:30.165883] Url Request Success
서울특별시 서초구 방배천로  21
[2018-08-03 00:49:30.259641] Url Request Success
서울특별시 마포구 어울마당로  94-13
[2018-08-03 00:49:30.369352] Url Request Success
서울특별시 서대문구 명물1길 16
[2018-08-03 00:49:30.469

In [18]:
with open('dish_place.json', 'w', encoding="utf-8") as fp:
    json.dump(final_dict, fp, ensure_ascii=False, indent="\t")